PART I

In [63]:
input = "input_small.txt"
with open(input, 'r') as infile:
    data = [l.strip() for l in infile.readlines()]
#Maintain a map of the current value of each identifier
#and a Boolean 'Flag' variable indicating whether the value has been changed by the current map
#Then we can just go through the map once and try to map everything that hasn't been mapped yet
#Then after each map, we have to reset this flag for every identifier
identifiers = [(int(n), False) for n in data[0].split()[1:]]

#Helper function to map source to destination
#Returns the new id alongside a boolean if the mapping was successful
def map_source_to_dest(dest_range_start, source_range_start, 
                       range_length, id_to_be_mapped):
    #If the value is outside the range, don't map
    if id_to_be_mapped < source_range_start or id_to_be_mapped >= source_range_start + range_length:
        return (id_to_be_mapped, False)
    else:
        return (dest_range_start + (id_to_be_mapped - source_range_start), True)
#I like to develop such functions test-driven.
#This means I write one or multiple test cases and code until all tests pass
#By writing this test I am already understanding better what exact problem I am trying to solve
#So here's the test I used to write this function
#('assert' means that if this statement isn't true, an error will be thrown) 
assert map_source_to_dest(12,15,3,16) == (13, True)


for line in data[1:]:
    if line == '':
        continue
    elif 'map:' in line:
        #Reset all flags
        identifiers = [(val, False) for (val, _) in identifiers]
    else:
        #I know this always has three elements
        dest_range_start, source_range_start, range_length = [int(x) for x in line.split()]
        identifiers = [map_source_to_dest(
            dest_range_start, source_range_start, range_length, identifier)
                       if not has_been_mapped else (identifier, has_been_mapped)
                       for (identifier, has_been_mapped) in identifiers]
print(min(identifier for (identifier, _) in identifiers))

35


PART II:

In [64]:
data

['seeds: 79 14 55 13',
 '',
 'seed-to-soil map:',
 '50 98 2',
 '52 50 48',
 '',
 'soil-to-fertilizer map:',
 '0 15 37',
 '37 52 2',
 '39 0 15',
 '',
 'fertilizer-to-water map:',
 '49 53 8',
 '0 11 42',
 '42 0 7',
 '57 7 4',
 '',
 'water-to-light map:',
 '88 18 7',
 '18 25 70',
 '',
 'light-to-temperature map:',
 '45 77 23',
 '81 45 19',
 '68 64 13',
 '',
 'temperature-to-humidity map:',
 '0 69 1',
 '1 0 69',
 '',
 'humidity-to-location map:',
 '60 56 37',
 '56 93 4']

Case Discrimination:
- Seed Range completely within map: Apply mapping to id start
- Seed Range completely not within map: Do nothing
- Seed Range overlaps on the left/right: Split into two lists, add value to the overlapping part and do nothing to the other part

Test-Driven Development will really be of help here!

In [90]:
def map_source_range_to_dest_range(dest_range_start, source_range_start, range_length,
                                   input_range_start, input_range_length):
    if input_range_start >= source_range_start:
        if input_range_start + input_range_length <= source_range_start + range_length:
            #Input is completely within the map
            return [(dest_range_start + (input_range_start - source_range_start), input_range_length, True)]
        elif input_range_start < source_range_start + range_length:
            #Input overlaps on the big end of the map
            first_range = source_range_start + range_length - input_range_start
            second_range = input_range_length - first_range
            return [(dest_range_start + (input_range_start - source_range_start), first_range, True),
                   (source_range_start + range_length, second_range, False)]
        else:
            #No overlap, input is too big for the map
            return [(input_range_start, input_range_length, False)]
    else:
        if input_range_start + input_range_length >= source_range_start:
            #Input overlaps on the small end of the map
            first_range = source_range_start - input_range_start
            second_range = input_range_length - first_range
            return [(input_range_start, first_range, False), (dest_range_start, second_range, True)]
        else:
            #No overlap, input is too small for the map
            return [(input_range_start, input_range_length, False)]



assert map_source_range_to_dest_range(42, 5, 10, 7, 4) == [(44, 4, True)]
assert map_source_range_to_dest_range(42, 5, 10, 1, 2) == [(1, 2, False)]
assert map_source_range_to_dest_range(42, 5, 10, 16, 2) == [(16, 2, False)]
assert map_source_range_to_dest_range(42, 5, 10, 14, 4) == [(51, 1, True), (15,3, False)]
assert map_source_range_to_dest_range(42, 5, 10, 4, 4) == [(4, 1, False), (42, 3, True)]


In [127]:
input = "input.txt"
with open(input, 'r') as infile:
    data = [l.strip() for l in infile.readlines()]
seed_line = data[0].split()[1:]
identifier_ranges = [(int(seed_line[i]), int(seed_line[i+1]), False) for i in range(0, len(seed_line), 2)]

for line in data[1:]:
    if line == '':
        continue
    elif 'map:' in line:
        identifier_ranges = [(val1, val2, False) for (val1, val2, _) in identifier_ranges]
    else:
        dest_range_start, source_range_start, range_length = [int(x) for x in line.split()]
        new_identifier_ranges = []
        for (input_range_start, input_range_length, has_been_mapped) in identifier_ranges:
            if has_been_mapped:
                new_identifier_ranges.append((input_range_start, input_range_length, has_been_mapped))
            else:
                new_identifier_ranges += map_source_range_to_dest_range(
                    dest_range_start, source_range_start, range_length, input_range_start, input_range_length)
        identifier_ranges = new_identifier_ranges

print(min(identifier for (identifier, _, _) in identifier_ranges))

125742456
